# Add voxel counts in annotation volume to an anatomical ontology file
Using the original annotation volume (AV) and anatomical ontology (AO) file of the mouse brain by Allen Institute for Brain Science (AIBS), this notebook checks voxel counts (VC) for each brain structure in AV, and adds them to AO file.

Prepare directories 'data', 'gene_data', 'fiber_data' and 'figs' at a working directory. Put the original files by AIBS, '1.json' and 'annotation_100.nrrd', at 'data' folder. Output files are saved in the above folders.

- input files
    - annotation_100.nrrd: The original AV by AIBS
    - 1.json: The original AO file by AIBS
- output file
    - 1_VC.json: AO file with voxel counts for each brain structure

# Set variables

In [99]:
dir_data = 'data'
fn_input_AV = 'annotation_100.nrrd' # filename (fn) for annotation volume
fn_input_AO = '1.json' # fn for anatomical ontology
fn_output_AO = '1_VC.json' # fn for anatomical ontology with voxel counts

In [98]:
import os
import nrrd
import numpy as np
import pandas as pd
import json
import copy
from collections import OrderedDict
from jsonpath_rw import jsonpath, parse

# Load data

In [100]:
AV, Header = nrrd.read(os.path.join(dir_data, fn_input_AV))

ID_unique = np.unique(AV)
IDvc_list = [[i, np.sum(AV == i)] for i in ID_unique]
IDvc = pd.DataFrame(IDvc_list, columns=['ID', 'voxel_count'])

with open(os.path.join(dir_data, fn_input_AO)) as f:
    df = json.load(f, object_pairs_hook=OrderedDict)

# Add voxel-counts to an annotation ontology file

In [101]:
def Add_VC_to_AO(match_id, match_fullpath):
    global df_VC
    global IDvc
    if IDvc['ID'].isin([match_id]).any():
        temp_voxelcount = IDvc.loc[IDvc['ID'] == match_id, 'voxel_count'].tolist()[0]
    else:
        temp_voxelcount = None
    exec("df_VC['msg'][0]"+\
         str(match_fullpath).replace('.','')\
          .replace('children', "['children']").replace('id', '')+\
         "['voxel_count'] = "+ str(temp_voxelcount))
    return

In [102]:
jsonpath_expr = parse('$..id')
df_VC = copy.deepcopy(df)
[Add_VC_to_AO(match.value, match.full_path) \
 for match in jsonpath_expr.find(df_VC['msg'][0])];

# Save AO with voxel counts

In [103]:
with open(os.path.join(dir_data, fn_output_AO), mode='w') as fw:
    json.dump(df_VC, fw, indent=4)

# Check data

In [104]:
print("Data type of the annotation volume: ", AV.dtype) # uint32
print("# unique ID: ", np.unique(AV).size) # There is 670 unique ID in AV
print('# total voxel: ', np.size(AV)) # 1203840 voxels
print('# brain voxel: ', np.sum(AV!=0)) # 505359
print('# outside brain voxel: ', np.sum(AV==0)) # 698481
IDvc.head()

Data type of the annotation volume:  uint32
# unique ID:  1098
# total voxel:  1203840
# brain voxel:  501770
# outside brain voxel:  702070


,ID,voxel_count
0,0,702070
1,1,54
2,2,63
3,7,550
4,9,115
